In [1]:
from agential.cog.functional.critic import _prompt_agent, _prompt_critic
from agential.cog.prompts.critic import (
    CRITIC_INSTRUCTION_HOTPOTQA, 
    HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC
)
from langchain.prompts import PromptTemplate
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.tools.google_serper.tool import GoogleSerperRun 

search = GoogleSerperRun(api_wrapper=GoogleSerperAPIWrapper())

In [2]:
from langchain_community.chat_models.openai import ChatOpenAI

import dotenv

dotenv.load_dotenv()

llm = ChatOpenAI()

question = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring'

answer = _prompt_agent(
    llm=llm, 
    keys={
        "question": question, 
        "examples": HOTPOTQA_FEWSHOT_EXAMPLES_COT
    }, 
    prompt_template=CRITIC_INSTRUCTION_HOTPOTQA
)

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


<PROMPT IN =======================================================================>
Q: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
A: Let's think step by step. The eastern sector of Colorado orogeny extends into the High Plains. High Plains rise in elevation from around 1,800 to 7,000 ft. So the answer is: 1,800 to 7,000 ft.

Q: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
A: Let's think step by step. Milhouse was named after U.S. president Richard Nixon. So the answer is: Richard Nixon.

Q: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
A: Let's think step by step. Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture. So the answer is: The Saimaa Gesture.

Q: What profes

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


<PROMPT OUT =======================================================================>
Let's break it down step by step. The person described was once considered the best kickboxer in the world, but has been involved in controversies for unsportsmanlike conduct and crimes of violence. This description matches the career and controversies of kickboxer Badr Hari. So the answer is: Badr Hari.
<PROMPT OUT =======================================================================>


In [3]:
print(answer)

Let's break it down step by step. The person described was once considered the best kickboxer in the world, but has been involved in controversies for unsportsmanlike conduct and crimes of violence. This description matches the career and controversies of kickboxer Badr Hari. So the answer is: Badr Hari.


In [96]:
critic_prompt = """{examples}
(END OF EXAMPLES)

Question: {question}
Proposed Answer: {answer}

What's the problem with the above answer?

1. Plausibility:
"""



critic_prompt = critic_prompt.format(
    **{    
        "examples": HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC,
        "question": question,
        "answer": answer
    }
)

In [144]:
import lmql

@lmql.query(model="openai/gpt-3.5-turbo", max_len=5000)
async def criticize(critic_prompt):
    '''lmql
    "{critic_prompt}"
    "[RESPONSE]"
    "2. Truthfulness:"
    "Let's search the question on google:"
    "> Search Query: [QUERY]" where STOPS_AT(QUERY, "> Evidence:")
    '''

o = await criticize(critic_prompt)

In [145]:
print(o.prompt)

Question: Serianna is a band of what genre that combines elements of heavy metal and hardcore punk?
Proposed Answer: Let's think step by step. Serianna is a band of metalcore genre. Metalcore is a subgenre of heavy metal and hardcore punk. So Serianna is a band of heavy metal and hardcore punk. So the answer is: heavy metal and hardcore punk.

What's the problem with the above answer?

1. Plausibility:

The question asks for the genre that combines elements of heavy metal and hardcore punk, and the answer is "heavy metal and hardcore punk", simply repeat the question. So it's not plausible.

2. Truthfulness:

Let's search the question in google:
> Search Query: Serianna is a band of what genre that combines elements of heavy metal and hardcore punk? site: wikipedia.org
> Evidence: [Metalcore - Wikipedia] Metalcore is a fusion music genre that combines elements of extreme metal and hardcore punk.

The evidence suggests that metalcore is a genre that combines elements of extreme metal an

In [79]:
from guidance import models, select, gen
llm = models.OpenAI("gpt-3.5-turbo-instruct")

In [81]:
stop_phrases_1 = ["> Search Query:", "most possible answer:"]
stop_phrase_2 = "> Evidence:"

out = llm + \
    critic_prompt + \
    gen(name="action_type", save_stop_text=True, stop=stop_phrases_1) + \
    gen(name="search_query", stop=stop_phrase_2)

query = out['search_query'].lstrip(stop_phrases_1[0])
evidence = search(query)

In [55]:
out_1 = llm + \
    str(out) + \
    stop_phrase_2 + evidence + \
    gen(name="something?")

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4516 tokens (3516 in your prompt; 1000 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}